In [1]:
import numpy as np
from sklearn import preprocessing


In [2]:
raw_csv_data = np.loadtxt("Audiobooks_data.csv",delimiter=',') 
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all =raw_csv_data[:,-1]

## Balance the dataset

In [3]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []


for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_targets_counter+=1
        if zero_targets_counter>num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_with_equal_priors = np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors = np.delete(targets_all,indices_to_remove,axis=0)

## Standardize the data

In [4]:
scaled_inputs = preprocessing.scale(unscaled_inputs_with_equal_priors)

## Shuffle the data

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]


## Split the data into train, validation and test set

In [32]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = int(0.1*samples_count)

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets),train_samples_count,np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets),validation_samples_count,np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets),test_samples_count,np.sum(test_targets)/test_samples_count)

1783.0 3579 0.4981838502374965
234.0 447 0.5234899328859061
220.0 447 0.49217002237136465


## Save the datasets in *.npz format

In [33]:
np.savez("Audiobooks_data_train",inputs=train_inputs,targets=train_targets)
np.savez("Audiobooks_data_validation",inputs=validation_inputs,targets=validation_targets)
np.savez("Audiobooks_data_test",inputs=test_inputs,targets=test_targets)

## Creating ML algorithm

In [34]:
import tensorflow as tf

## Data

In [35]:
npz1 = np.load("Audiobooks_data_train.npz")

train_inputs = npz1['inputs'].astype(float)
train_targets = npz1['targets'].astype(int)

npz2 = np.load("Audiobooks_data_validation.npz")
validation_inputs, validation_targets =  npz2['inputs'].astype(float),npz2['targets'].astype(int)

npz3 = np.load("Audiobooks_data_test.npz")
test_inputs,test_targets = npz3["inputs"].astype(float),npz3["targets"].astype(int)

## Model

## Outline,loss, early stopping and training


In [36]:
input_size = 10
output_size = 2
hidden_layer_size = 100

model = tf.keras.Sequential([
                            
                            tf.keras.layers.Dense(hidden_layer_size,activation = "relu"),
                            tf.keras.layers.Dense(hidden_layer_size,activation = "relu"),
                            tf.keras.layers.Dense(output_size,activation = "softmax")
    
    
    
    
])

## Choose optimizer and loss function

In [37]:
model.compile(optimizer ="adam", loss = "sparse_categorical_crossentropy",metrics=["accuracy"])  
##sparse_categorical_crossentropy is used to ensure that the targets are one-hot encoded


In [38]:
batch_size = 50
max_epochs = 10000
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,train_targets,
          batch_size=batch_size,epochs=max_epochs,callbacks=[early_stopping],
          validation_data=(validation_inputs,validation_targets),
         verbose=2)

Epoch 1/10000
72/72 - 1s - loss: 0.4820 - accuracy: 0.7499 - val_loss: 0.4305 - val_accuracy: 0.7606 - 912ms/epoch - 13ms/step
Epoch 2/10000
72/72 - 0s - loss: 0.3859 - accuracy: 0.7899 - val_loss: 0.3855 - val_accuracy: 0.8009 - 143ms/epoch - 2ms/step
Epoch 3/10000
72/72 - 0s - loss: 0.3597 - accuracy: 0.8089 - val_loss: 0.3699 - val_accuracy: 0.7987 - 158ms/epoch - 2ms/step
Epoch 4/10000
72/72 - 0s - loss: 0.3510 - accuracy: 0.8055 - val_loss: 0.3595 - val_accuracy: 0.8009 - 149ms/epoch - 2ms/step
Epoch 5/10000
72/72 - 0s - loss: 0.3476 - accuracy: 0.8128 - val_loss: 0.3632 - val_accuracy: 0.8143 - 149ms/epoch - 2ms/step
Epoch 6/10000
72/72 - 0s - loss: 0.3364 - accuracy: 0.8150 - val_loss: 0.3563 - val_accuracy: 0.8031 - 148ms/epoch - 2ms/step
Epoch 7/10000
72/72 - 0s - loss: 0.3346 - accuracy: 0.8120 - val_loss: 0.3516 - val_accuracy: 0.8277 - 149ms/epoch - 2ms/step
Epoch 8/10000
72/72 - 0s - loss: 0.3360 - accuracy: 0.8122 - val_loss: 0.3477 - val_accuracy: 0.8166 - 148ms/epoch - 

## Test

In [39]:
test_loss,test_accuracy = model.evaluate(test_inputs,test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3412 - accuracy: 0.8326
